In [1]:
import pandas as pd
import numpy as np
import os
from sentence_transformers import SentenceTransformer, CrossEncoder, util
import torch
import hnswlib
from transformers import AutoModel

if os.path.isfile("rag_instruct.json"): 
    df = pd.read_json("rag_instruct.json")
else:
    df = pd.read_json("hf://datasets/FreedomIntelligence/RAG-Instruct/rag_instruct.json")

documents = df['documents']

In [2]:
semb_model = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')
xenc_model = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')
#semb_model.to('cuda')

In [3]:
corpus_embeddings = semb_model.encode(documents, convert_to_tensor=True, show_progress_bar=True)


Batches:   0%|          | 0/1267 [00:00<?, ?it/s]

In [4]:
size_corpus = len(corpus_embeddings)
print("Size of corpus: ", size_corpus)
index = hnswlib.Index(space='cosine', dim=corpus_embeddings.size(1))

Size of corpus:  40541


In [5]:
# Define hnswlib index path
index_path = "./hnswlib.index"

# Load index if available
if os.path.exists(index_path):
    print("Loading index...")
    index.load_index(index_path)
# Else index data collection
else:
    # Initialise the index
    print("Start creating HNSWLIB index")
    index.init_index(max_elements=corpus_embeddings.size(0), ef_construction=400, M=64)
    #  Compute the HNSWLIB index (it may take a while)
    index.add_items(corpus_embeddings.cpu(), list(range(len(corpus_embeddings))))
    # Save the index to a file for future loading
    print("Saving index to:", index_path)
    index.save_index(index_path)

Start creating HNSWLIB index
Saving index to: ./hnswlib.index


In [6]:
# function to get the related docs
def get_related_docs(query, k=3):
    query_embedding = semb_model.encode(query, convert_to_tensor=True)
    corpus_ids, _ = index.knn_query(query_embedding.cpu(), k=k)

    model_inputs = [(query, str(documents[idx])) for idx in corpus_ids[0]]
    cross_scores = xenc_model.predict(model_inputs)
    send_to_LLM = ""
    positive_docs = [documents[corpus_ids[0][idx]] for idx in np.argsort(-cross_scores) if cross_scores[idx] > 0]

    if len(positive_docs) > 1:
        for i, doc in enumerate(positive_docs):
            send_to_LLM += f"Document {i+1}:\n\n"
            # Convert the list 'doc' to a string before concatenating
            send_to_LLM += str(doc) + "\n"
    elif len(positive_docs) == 1:
        # Convert the list to a string if there's only one document
        send_to_LLM = str(positive_docs[0])

    else:
        # If no positive scores, take the top 2 negative scores
        negative_docs = []
        for idx in np.argsort(-cross_scores)[:2]: # Take the top 2 indices based on sorted scores
            negative_docs.append(documents[corpus_ids[0][idx]])

        if len(negative_docs) > 1:
            for i, doc in enumerate(negative_docs):
                send_to_LLM += f"Document {i+1}:\n"
                send_to_LLM += str(doc) + "\n\n"
        elif len(negative_docs) == 1:
            send_to_LLM = str(negative_docs[0])

    return send_to_LLM



In [7]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,  # or bfloat16 if supported
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

# Load model with quantization
llm_model = AutoModelForCausalLM.from_pretrained(
    "AITeamVN/Vi-Qwen2-3B-RAG",
    quantization_config=quant_config,
    device_map="auto"
)

llm_tokenizer = AutoTokenizer.from_pretrained("AITeamVN/Vi-Qwen2-3B-RAG")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
def generate_response(query):
    context_docs = get_related_docs(query)

    prompt = f"Given this context: \n{context_docs} \n\nPlease answer the question: {query}.\n\nAnswer:\n"

    inputs = llm_tokenizer(prompt, return_tensors="pt")

    with torch.no_grad():
        outputs = llm_model.generate(
            input_ids=inputs["input_ids"].to(llm_model.device),
            attention_mask=inputs["attention_mask"].to(llm_model.device),
            max_new_tokens=200,
            temperature=0.7,
            top_p=0.9,
            do_sample=True,
            pad_token_id=llm_tokenizer.eos_token_id
        )

    # Decode and print result
    answer = llm_tokenizer.decode(outputs[0], skip_special_tokens=True)
    print("\n=== Generated Answer ===\n")
    return answer.split("Answer:")[-1].strip()

In [9]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,  # or bfloat16 if supported
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

# Load model with quantization
model = AutoModelForCausalLM.from_pretrained(
    "AITeamVN/Vi-Qwen2-3B-RAG",
    quantization_config=quant_config,
    device_map="auto"
)

tokenizer = AutoTokenizer.from_pretrained("AITeamVN/Vi-Qwen2-3B-RAG")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [10]:
import sounddevice as sd
import numpy as np
import webrtcvad
import whisper
import collections
import struct
import re
from transformers import WhisperForConditionalGeneration, WhisperProcessor, BitsAndBytesConfig
import torch

SAMPLE_RATE = 16000
FRAME_DURATION_MS = 30  # ms
FRAME_SIZE = int(SAMPLE_RATE * FRAME_DURATION_MS / 1000)
CHANNELS = 1
VAD_AGGRESSIVENESS = 2  # 0–3: higher = more aggressive
MAX_SILENCE_SECONDS = 1.0

# === Load Whisper ===
model_id = "openai/whisper-base"  # You can also try "small", "medium", etc.

processor = WhisperProcessor.from_pretrained(model_id)
model = WhisperForConditionalGeneration.from_pretrained(
    model_id,
    device_map="auto",
    quantization_config=quant_config
)

# === Setup VAD ===
vad = webrtcvad.Vad(VAD_AGGRESSIVENESS) 



In [ ]:
GOODBYE_PATTERNS = [
    r"\b(bye|goodbye|see you|exit|quit|farewell)\b",
    r"talk to you later",
    r"that's all",
    r"that's it"
]

def float32_to_int16(audio):
    return (audio * 32767).astype(np.int16)

def is_speech(frame_bytes):
    return vad.is_speech(frame_bytes, SAMPLE_RATE)

def is_goodbye(text):
    for pattern in GOODBYE_PATTERNS:
        if re.search(pattern, text, re.IGNORECASE):
            return True
    return False

def record_until_silence():
    print("Listening...")

    buffer = []
    silence_buffer = collections.deque(maxlen=int(MAX_SILENCE_SECONDS * 1000 / FRAME_DURATION_MS))
    stream = sd.InputStream(samplerate=SAMPLE_RATE, channels=CHANNELS, dtype='float32', blocksize=FRAME_SIZE)

    with stream:
        while True:
            audio_chunk, _ = stream.read(FRAME_SIZE)
            audio_chunk = audio_chunk.flatten()
            audio_int16 = float32_to_int16(audio_chunk)
            frame_bytes = struct.pack(f"{len(audio_int16)}h", *audio_int16)

            if is_speech(frame_bytes):
                buffer.append(audio_chunk)
                silence_buffer.clear()
            else:
                silence_buffer.append(audio_chunk)
                if len(silence_buffer) == silence_buffer.maxlen and len(buffer) > 0:
                    print("Silence detected, stopping...")
                    break

    full_audio = np.concatenate(buffer)
    return full_audio


def transcribe_audio_array(audio_array: np.ndarray, sampling_rate: int):
    if audio_array.ndim > 1:
        audio_array = audio_array.mean(axis=1)  # convert stereo to mono

    # Convert to tensor
    waveform = torch.tensor(audio_array, dtype=torch.float32)

    # Resample if needed
    if sampling_rate != 16000:
        import torchaudio
        waveform = torchaudio.functional.resample(waveform, orig_freq=sampling_rate, new_freq=16000)

    # Whisper expects float32 here — processor handles float32, model input will be auto-cast later
    inputs = processor(waveform, sampling_rate=16000, return_tensors="pt")
    input_features = inputs.input_features.to(model.device, dtype=torch.float16)  # ✅ cast to float16

    with torch.no_grad():
        predicted_ids = model.generate(input_features)
    transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]
    return transcription


# === Conversation Loop ===
def start_conversation():
    print("Start speaking. Say 'goodbye' to end the conversation.")
    while True:
        audio = record_until_silence()
        if len(audio) == 0:
            continue  # skip empty audio

        print("Transcribing...")
        
        # Decode
        text = transcribe_audio_array(audio, sampling_rate=SAMPLE_RATE)[:50]

        print(f"You have said: {text}")
        if is_goodbye(text):
            print("Goodbye detected. Ending conversation.")
            break

        generated_answer = generate_response(text)
        print(f"AI: {generated_answer}")



In [12]:
start_conversation()

Start speaking. Say 'goodbye' to end the conversation.
Listening...


Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Silence detected, stopping...
Transcribing...

=== Generated Answer ===

AI: Photosynthesis is a process used by plants and other organisms to convert light energy into chemical energy that can later be released to fuel the organisms' activities. This chemical energy is stored in carbohydrate molecules, such as sugars, which are synthesized from carbon dioxide and water. Oxygen is also released as a waste product in most cases.

In more detail:
- Photosynthesis occurs in the chloroplasts of plant cells.
- Water (HO) and carbon dioxide (CO₂) are used as raw materials.
- Energy from light is captured and converted into chemical energy, primarily stored in the form of glucose.
- The process can be divided into two main stages: the light-dependent reactions and the light-independent reactions (Calvin cycle).
- In the light-dependent reactions, light energy is used to split water molecules, releasing oxygen and creating ATP (adenosine triphosphate) and NADPH (nicotinamide adenine dinucleoti